Import Libraries

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from glob import glob
from tqdm import tqdm

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms


import segmentation_models_pytorch as smp


import warnings
warnings.filterwarnings("ignore")


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}")

✅ Using device: cpu


Data Path & Dataset Class

In [2]:
train_image_dir = r"C:\Users\ghwns\HJ_git\CV-Projects\urban-scene-segmentation\Dataset\Cityspaces\images\train"
train_mask_dir  = r"C:\Users\ghwns\HJ_git\CV-Projects\urban-scene-segmentation\Dataset\Cityspaces\gtFine\train"

val_image_dir = r"C:\Users\ghwns\HJ_git\CV-Projects\urban-scene-segmentation\Dataset\Cityspaces\images\val"
val_mask_dir  = r"C:\Users\ghwns\HJ_git\CV-Projects\urban-scene-segmentation\Dataset\Cityspaces\gtFine\val"



class CityscapesDataset(Dataset):
    def __init__(self, image_dir, mask_dir, resize=(256, 256)):
        self.image_paths = sorted(glob(os.path.join(image_dir, "*", "*.png")))
        self.mask_paths  = [p.replace("images", "gtFine").replace("_leftImg8bit", "_gtFine_labelIds") for p in self.image_paths]

        self.resize = resize
        self.to_tensor = transforms.ToTensor()
        self.resize_img = transforms.Resize(resize)
        self.resize_mask = transforms.Resize(resize, interpolation=Image.NEAREST)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        mask  = Image.open(self.mask_paths[idx])

        image = self.resize_img(image)
        mask  = self.resize_mask(mask)

        image = self.to_tensor(image)
        mask  = torch.from_numpy(np.array(mask)).long()

        return image, mask

In [3]:
train_dataset = CityscapesDataset(
    image_dir=train_image_dir,
    mask_dir=train_mask_dir,
    resize=(256, 256)
)

val_dataset = CityscapesDataset(
    image_dir=val_image_dir,
    mask_dir=val_mask_dir,
    resize=(256, 256)
)


train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=0)

print(f"✅ Train samples: {len(train_dataset)} | Val samples: {len(val_dataset)}")

✅ Train samples: 2975 | Val samples: 500


Define Model & Loss + Optimizer

In [4]:
# 모델 정의
model = smp.Unet(
    encoder_name="resnet34",        
    encoder_weights="imagenet",    
    in_channels=3,                
    classes=34                      
)


model = model.to(device)


criterion = nn.CrossEntropyLoss()


optimizer = optim.Adam(model.parameters(), lr=1e-4)

print("✅ Model, loss, optimizer initialized.")

✅ Model, loss, optimizer initialized.


Training loop

In [ ]:
from tqdm import tqdm

num_epochs = 5  

for epoch in range(num_epochs):
    model.train()
    train_loss = 0

    print(f"\n🔁 Epoch {epoch+1}/{num_epochs}")

    # --- Train loop with tqdm ---
    train_loader_tqdm = tqdm(train_loader, desc="🔧 Training", leave=False)
    for images, masks in train_loader_tqdm:
        images, masks = images.to(device), masks.to(device)

        outputs = model(images)
        loss = criterion(outputs, masks)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_loader_tqdm.set_postfix(loss=loss.item())

    # --- Validation loop with tqdm ---
    model.eval()
    val_loss = 0

    val_loader_tqdm = tqdm(val_loader, desc="🔍 Validating", leave=False)
    with torch.no_grad():
        for images, masks in val_loader_tqdm:
            images, masks = images.to(device), masks.to(device)

            outputs = model(images)
            loss = criterion(outputs, masks)

            val_loss += loss.item()
            val_loader_tqdm.set_postfix(val_loss=loss.item())

    # --- Epoch summary ---
    print(f"✅ Epoch [{epoch+1}/{num_epochs}] | 🟢 Train Loss: {train_loss:.4f} | 🔵 Val Loss: {val_loss:.4f}")


🔁 Epoch 1/5


🔧 Training:  61%|██████████████████████████████████▎                     | 456/744 [28:22<15:35,  3.25s/it, loss=0.89]